In [1]:
import concurrent.futures
from multiprocessing.pool import ThreadPool
import benchmark
import os
import syfco
import utils
import csv
import math
import json
from utils import write_file

{'StatusCode': 0} b'module fsm(upd, in_0, in_1, in_2, in_3, out_0, out_1, out_2, out_3);\n  input upd;\n  input in_0;\n  input in_1;\n  input in_2;\n  input in_3;\n  output out_0;\n  output out_1;\n  output out_2;\n  output out_3;\n  reg [3:0] state;\n\n  assign out_0 = ((state == 2) && !(!in_0 && upd) || (state == 9) && !(!in_0 && upd) || (state == 6) && (in_0 && upd) || (state == 13) && (in_0 && upd) || (state == 4) && (in_0 && upd) || (state == 12) && (in_0 && upd) || (state == 10) && !(!in_0 && upd) || (state == 11) && (in_0 && upd) || (state == 1) && !(!in_0 && upd) || (state == 7) && !(!in_0 && upd) || (state == 15) && (in_0 && upd) || (state == 8) && !(!in_0 && upd) || (state == 3) && !(!in_0 && upd) || (state == 5) && (in_0 && upd) || (state == 0) && (in_0 && upd) || (state == 14) && !(!in_0 && upd)) ? 1 : 0;\n  assign out_1 = ((state == 2) && !(!in_1 && upd) || (state == 9) && !(!in_1 && upd) || (state == 6) && !(!in_1 && upd) || (state == 13) && !(!in_1 && upd) || (state == 4

In [2]:
def crawlDir(path):
  filtered_files = []
  for root, subdirs, files in os.walk(path):
    for file in filter(lambda s: s.endswith(".tlsf"), files):
      full = os.path.join(root, file)
      params = syfco.parameters(utils.read_file(full))
      if len(params) == 1:
        filtered_files.append((full, params[0]))
  return filtered_files
tlsf_files = crawlDir("../../benchmarks/tlsf/")

In [7]:
import strix
from utils import read_file
def get_benchmarks(files) -> list[benchmark.Benchmark]:
   bms = []
   for (file, param) in files:
      (path, name) = os.path.split(file)
      (basename, _, _) = name.rpartition(".")
      csvpath = os.path.join(path, basename + ".csv")
      if not os.path.isfile(csvpath):
         continue
      with open(csvpath) as csvfile:
         spamreader = csv.DictReader(csvfile, dialect="unix")
         rows = list(spamreader)
         if any(row["status"] == "unrealizable" for row in rows): # are any of the benchmarks unrealizable:
            continue
         realizable_values = sorted(int(row[param]) for row in rows if row["status"] == "realizable")

         if len(realizable_values) < 2:
               print("Not enough realizable examples for " + basename)
               continue # not synthesizable for any value of the parameter (or just one)
         
         maxval = realizable_values[-1]
         maxpow2 = pow(2, (maxval).bit_length() - 1)
         while (True):
            try:
               print("Synthesizing " + basename)
               strix.synthesize(
                  read_file(file), # spec
                  overwrite_params={param: maxpow2 if maxpow2 >= 4 else maxval},
                  timeout = 60
               )
               if maxpow2 < 4:
                  examples = realizable_values[-2:] # two largest values
                  gen = maxval + 1
               else:
                  examples = [maxpow2 // 2, maxpow2]
                  gen = 2 * maxpow2
               bms.append(benchmark.Benchmark({
                  "name": basename,
                  "params": [param],
                  "specification": os.path.abspath(file),
                  "generate": {param: gen},
                  "implementations": [{"params": {param: n}} for n in examples]
               },"../../benchmarks/tlsf/"))
               break
            except:
               print("Timed out: " + basename)
               if maxpow2 >= 8:
                  maxpow2 = maxpow2 // 2
               elif maxpow2 >= 4:
                  maxpow2 = maxpow2 // 2
                  realizable_values = [v for v in realizable_values if v < maxpow2]
                  maxval = realizable_values[-1]
               else:
                  realizable_values = realizable_values[:-1] #remove biggest parameter
                  maxval = realizable_values[-1]
                  if len(realizable_values) < 2:
                     print("Not enough realizable examples for " + basename)
                     break # not synthesizable (under 1m) for any value of the parameter (or just one)
               continue            
   return bms
bms = get_benchmarks(tlsf_files)
with open("test.json", "w") as f:
   json.dump(bms, cls=benchmark.BenchmarkEncoder, fp=f, indent=2)
print(bms)

Not enough realizable examples for amba_case_study
Synthesizing amba_decomposed_arbiter
Timed out: amba_decomposed_arbiter
Synthesizing amba_decomposed_arbiter
Synthesizing amba_decomposed_encode
Timed out: amba_decomposed_encode
Synthesizing amba_decomposed_encode
Synthesizing amba_decomposed_lock
Timed out: amba_decomposed_lock
Synthesizing amba_decomposed_lock
Timed out: amba_decomposed_lock
Synthesizing amba_decomposed_lock
Timed out: amba_decomposed_lock
Synthesizing amba_decomposed_lock
Not enough realizable examples for example10
Not enough realizable examples for example10a
Not enough realizable examples for example7
Not enough realizable examples for example7a
Not enough realizable examples for example8
Not enough realizable examples for example9
Not enough realizable examples for abcg_arbiter
Not enough realizable examples for arbiter
Not enough realizable examples for arbiter_with_buffer
Not enough realizable examples for arbiter_with_cancel
Synthesizing collector_v1
Synthes

```
Not enough realizable examples for amba_case_study
Synthesizing amba_decomposed_arbiter
Timed out: amba_decomposed_arbiter
Synthesizing amba_decomposed_arbiter
Synthesizing amba_decomposed_encode
Timed out: amba_decomposed_encode
Synthesizing amba_decomposed_encode
Synthesizing amba_decomposed_lock
Timed out: amba_decomposed_lock
Synthesizing amba_decomposed_lock
Timed out: amba_decomposed_lock
Synthesizing amba_decomposed_lock
Timed out: amba_decomposed_lock
Synthesizing amba_decomposed_lock
Not enough realizable examples for example10
Not enough realizable examples for example10a
Not enough realizable examples for example7
Not enough realizable examples for example7a
Not enough realizable examples for example8
Not enough realizable examples for example9
Not enough realizable examples for abcg_arbiter
Not enough realizable examples for arbiter
Not enough realizable examples for arbiter_with_buffer
Not enough realizable examples for arbiter_with_cancel
Synthesizing collector_v1
Synthesizing collector_v2
Synthesizing collector_v3
Synthesizing detector
Synthesizing full_arbiter
Timed out: full_arbiter
Synthesizing full_arbiter
Not enough realizable examples for full_arbiter_enc
Not enough realizable examples for generalized_buffer
Not enough realizable examples for lift
Not enough realizable examples for lift_unary_enc
Not enough realizable examples for lift_wrong_physics_unreal
Synthesizing load_balancer
Timed out: load_balancer
Synthesizing load_balancer
Synthesizing ltl2dba_alpha
Synthesizing ltl2dba_beta
Synthesizing ltl2dba_C2
Synthesizing ltl2dba_E
Synthesizing ltl2dba_Q
Timed out: ltl2dba_Q
Synthesizing ltl2dba_Q
Synthesizing ltl2dba_U1
Synthesizing mux
Timed out: mux
Synthesizing mux
Timed out: mux
Synthesizing mux
Timed out: mux
Synthesizing mux
Synthesizing narylatch
Synthesizing prioritized_arbiter
Synthesizing prioritized_arbiter_enc
Timed out: prioritized_arbiter_enc
Synthesizing prioritized_arbiter_enc
Synthesizing round_robin_arbiter
Synthesizing shift
Timed out: shift
Synthesizing shift
Timed out: shift
Synthesizing shift
Timed out: shift
Synthesizing shift
Timed out: shift
Synthesizing shift
Synthesizing simple_arbiter
Synthesizing simple_arbiter_enc
Timed out: simple_arbiter_enc
Synthesizing simple_arbiter_enc

In [9]:
timeouts = []
for bm in bms:
  try:
    benchmark.build_prompt(bm, mode="bosy")
  except e:
    print(e)
    print("timeout: " + bm.name)
    timeouts.append(bm)

NameError: name 'e' is not defined

In [23]:
import time
def test(x):
  time.sleep(5)
  print("Test()")

def update():
    try:
        with ThreadPool(1) as pool:
            result = pool.map_async(test, [()])
            for num in result.get(3):
                print(num)
    except (e):
        print(e)
update()

NameError: name 'e' is not defined

Test()
